In [1]:
import sys
import os
import numpy as np
import struct
import json
import datetime
import scipy
import scipy.signal
import matplotlib.pyplot as plt
%matplotlib inline

notebook_path = os.path.dirname(os.path.abspath("__file__"))
# the code path is two folders up from this notebook + /code
core_path = os.path.dirname(notebook_path)
basepath = os.path.dirname(os.path.dirname(notebook_path))

sys.path.append(core_path)
sys.path.append(basepath)

from core.readMDA import readMDA, get_Fs
from core.Tint_Matlab import int16toint8
from core.tetrode_conversion import convert_tetrode, is_tetrode, batch_basename_tetrodes, \
batch_add_tetrode_headers, get_tetrode_parameters, write_tetrode_header 
from core.convert_position import convert_position
from core.eeg_conversion import convert_eeg, get_eeg_channels
from core.utils import find_sub, session_datetime
from core.intan2mda import intan2mda, get_reref_data
from core.mdaSort import sort_intan
from core.set_conversion import convert_setfile, get_session_parameters
from core.rhd_utils import intan_scalar, tetrode_map, tintRef2intan, intanRef2tint
from core.intan_rhd_functions import rhd_duration, find_basename_files, read_data, read_header, \
get_data_limits, is_session_beginning, get_probe_name, get_ref_index
from core.intan_mountainsort import validate_session, convert_intan_mountainsort

# Parameters You Can Change

In [15]:
# INTERPOLATE
# interpolation=True  # if you want to interpolate, options: True or False, remember capital letter
interpolation=False
if interpolation:
    desired_Fs = int(48e3)  # Sampling Frequency you will interpolate to
else:
    desired_Fs = int(24e3)

# WHITEN
whiten = 'true'  # if you want to whiten or not, 'true' or 'false', lower case letters
# whiten = 'false'

# THRESHOLD
flip_sign = True # if you want to flip the signal (multiply by -1)
# remember if flip_sign is true then the negative troughs become peaks, so you should
# do detect_sign = 1
detect_interval = 10  # it will split up the data into segments of this value and find a peak/trough for each segment
detect_sign = 1  # 0 = positive and negative peaks, 1 = positive peaks, -1 = negative peaks

if whiten == 'true':
    detect_threshold = 3.5  # the threshold of the data, if whitened, data is normalized to standard deviations, so a value of 3
    # would mean 3 standard deviations away from baseline. If not whitened treat it like a Tint threshold, i.e. X bits. 
    # essentially bits vs standard deviations.
else:
    # you can grab a value from a set file with this animal, the parameter is called 'threshold', it is however in
    # 16bit, so divide the value by 256 to convert to 8bit, since the thresholding is in 8 bit.
    detect_threshold = 33
    
# BANDPASS
freq_min = 300  # min frequency to bandpass at
freq_max = 7000  # max freq to bandpass at

# EEG Settings

eeg_channels = 'first'  # this will save the 1st channel as an .eeg in each tetrode
# eeg_channels = 'all'  # this will save all the channels as their own .eeg file
# eeg_channels = [W, X, Y, Z]  # list of channels if you want to specify which ones to use

#MASK
mask=True

masked_chunk_size = None  # The amount of indices that you will segment for masking artifacts. 
# if you leave as None it will use a value of Fs/10 or Fs/20, I forget

mask_num_write_chunks = 100  # this is how many of the chunk segments will be written at the same time, mainly just for
# optimizing write speeds, just try to leave it as ~100k - 300k samples.

mask_threshold = 6  # number of standard deviations the Root of the Sum of the Squares (RSS) of all the segments 
# (of masked chunk size). If the RSS is above this threshold it will assume it is artifact, and set all values in this
# segment to zero.

# software re-ref PARAMETERS (essentially software re-referencing)
software_rereference = True  # if you want to remove the common signal (mean values) 
# between channels, set to True, otherwise False

# reref_method=None
reref_method = 'sd'  # methods of which to remove signal, 'sd' will caculate the standard deviation of every channel
# and choose the channels with the two lowest values and remove those signals from the rest (the 2nd lowest will be removed
# from the 1st lowest). Somewhat like what you guys do visually in Axona.

reref_channels = 'auto'  # below we have a dictionary of previously chosen values 
# depending on the mouse
# if set to auto, it will choose those values.

# reref_channels = [16, 9]  # if you wanted to just say which 
# channels to subtract from the rest, do it here, 
# reref_channels = None
# it will override the automatic stuff. Essentially look through .set files

clip_size = 50  # samples per spike, default 50

notch_filter = True  # if you want to add a notch. However, it is already filtered using freq_min, so this doesn't really help
# unless of course your freqmin is below 60 Hz, default False

positionSampleFreq = 50  # sampling frequency of position, default 50

pre_spike_samples = 15  # number of samples pre-threshold samples to take, default 10
post_spike_samples = 35  # number of post threshold samples to take, default 40

if pre_spike_samples + post_spike_samples != clip_size:
    raise ValueError(
        "the pre (%d) and post (%d) spike samples need to add up to the clip_size: %d." % (
            pre_spike_samples, post_spike_samples, int(clip_size)))

# Axona Artifact Rejection Criteria, I'd just leave these. They are in the manual
rejthreshtail = 43  #  I think, the latter 20-30% can't be above this value ( I think)
rejstart = 30  #
rejthreshupper = 100  # if 1st sample is above this value in bits, it will discard
rejthreshlower = -100  # if 1st sample is below this value in bits, it will discard

# The percentage of spikes to remove as outliers, this will make it so they don't make the majority of the 
# spikes look really small in amplitude
remove_outliers = True  # if False, it won't remove outliers, if True, it will.
remove_spike_percentage = 5  # percent value, default 1, haven't experimented much with this

remove_method = 'max'  # this will find the max of the peak values (or min if it's negative)
# and set that as the clipping value

clip_scalar = 1.05
# clip_scalar = 1  # this will multiply the clipping value found via the remove_method method, 
# and then scale by this value.

# feature parameters
num_features = 10
# max_num_clips_for_pca = 3000
max_num_clips_for_pca = 1000

# miscellaneous
self=None  # this is code jargin for object oriented programming, mainly used for GUI's, we don't need this
# just needs to be set in the function so I have it set to None.

In [16]:
# these are TINT channels, 
# these are 0-based so channel 0 here is channel 1 (or T1Ch1), 1 = T1Ch2, 
# channel 4 is T2Ch1
axona_refs = {
    'b6_august_18_1': [4, 3],  # 2a, 1d
    'b6_august_18_2': [4, 3],  # 2a, 1d
    'j20_sleep_1': [4, 3],  # 2a, 1d
    'j20_sleep_2': [4, 3],  # 2a, 1d
    'b6_sep_18_1': [4, 3],  # 2a, 1d
    'ANT1_2': [4, 3],  # 2a, 1d
    'NT_360a_2': [15, 6],  # 4d, 2c
    'NT_361a_2': [15, 6],  # 4d, 2c
    'march_19': [13, 3],  # 2a, 1d
    'NT_181': [4, 3],  # 2a, 1d
    'b6_j20n_76a_2': [15, 6],  # 4d, 2c
    'b6_j20n_82a_5': [15, 6],  # 4d, 2c
}

# Batch Analyzes

In [17]:
data_directory = r'E:\Apollo_D_Drive\data\VirtualMazeData'
# data_directory = r'E:\Apollo_D_Drive\data\MSData'
    
skip_dirs = ['Processed', 'RewardTrainingTrack', 'Other']
skip_mice = ['ANT1_2']

analyzed_directories = []

for root, dirs, files in os.walk(data_directory, topdown=False):
    
    if len(dirs) == 0:
        continue
    
    for dir_ in dirs:
        if dir_ not in skip_dirs:
            directory = os.path.join(root, dir_)
        else:
            print('Skipping the following directory: %s' % dir_)
            continue
        
        if directory in analyzed_directories:
            continue
        
        basename_files = [os.path.join(directory, file) for file in os.listdir(directory) if '.rhd' in file if is_session_beginning(os.path.join(directory, file))]

        if len(basename_files) == 0:
            continue

        print('--------------')
        print('There are %d .rhd sessions in this directory!' % len(basename_files))

        mouse = os.path.basename(os.path.dirname(directory))
        print('mouse: %s' % mouse)

        if mouse in skip_mice:
            print('Skipping the following mouse: %s' % mouse)
            continue
            
        # if 'b6_j20n_76a_2' not in mouse and '82a_5' not in mouse:
        #     print('mouseeeee %s' % mouse)
        #     print('Skipping the following mouse: %s' % mouse)
        #     continue
            
        for i, current_session in enumerate(basename_files):
            # grabs session files

            directory = os.path.dirname(current_session)

            tint_basename = os.path.basename(os.path.splitext(current_session)[0])
            tint_fullpath = os.path.join(directory, tint_basename)

            print('Analyzing file (%d/%d): %s' % (i+1, len(basename_files), tint_basename))

            output_basename = '%s_ms' % tint_fullpath

            session_valid = validate_session(current_session, output_basename, eeg_channels, self=self)

            if not session_valid:
                print('The following session has already been analyzed, or is missing required files: %s!' % os.path.basename(
                    tint_fullpath))
                continue

            rhd_session_file = os.path.splitext(os.path.basename(current_session))[0]

            rhd_basename = rhd_session_file[:find_sub(rhd_session_file, '_')[-2]]

            session_files = find_basename_files(rhd_basename, directory)

            rhd_session_fullfile = os.path.join(directory, rhd_session_file + '.rhd')

            # find the session with our rhd file in it
            session_files = [sub_list for sub_list in session_files if rhd_session_fullfile in sub_list][0]

            if type(session_files) != list:
                # if there is only one file in the list, the output will not be a list
                session_files = [session_files]

            # output files
            probe = get_probe_name(current_session)

            if mouse not in axona_refs.keys():
                for key in axona_refs.keys():
                    if key in session_files[0]:
                        mouse = key

            if reref_channels == 'auto':
                reref_channels = tintRef2intan(axona_refs[mouse], 
                                               tetrode_map, 
                                               probe)
                print('The following reref_channels were chosen: ', reref_channels)


            pos_filename = output_basename + '.pos'
            set_filename = tint_fullpath + '.set'
            bin_filename = tint_fullpath + '.bin'

            converted_set_filename = output_basename + '.set'
            # Process returned with non-zero exit code
            convert_intan_mountainsort(session_files, interpolation=interpolation, whiten=whiten, 
                                       detect_interval=detect_interval,
                                       detect_sign=detect_sign,  detect_threshold=detect_threshold, 
                                       freq_min=freq_min,
                                       freq_max=freq_max, mask_threshold=mask_threshold, 
                                       flip_sign=flip_sign,
                                       software_rereference=software_rereference, 
                                       reref_method=reref_method,
                                       reref_channels=reref_channels, 
                                       masked_chunk_size=masked_chunk_size,
                                       mask=mask,
                                       mask_num_write_chunks=mask_num_write_chunks, 
                                       clip_size=clip_size,
                                       notch_filter=notch_filter, 
                                       desired_Fs=desired_Fs, 
                                       positionSampleFreq=positionSampleFreq, 
                                       pre_spike_samples=pre_spike_samples, 
                                       post_spike_samples=post_spike_samples, 
                                       rejthreshtail=rejthreshtail, rejstart=rejstart,
                                       rejthreshupper=rejthreshupper, rejthreshlower=rejthreshlower,
                                       remove_spike_percentage=remove_spike_percentage, 
                                       remove_outliers=remove_outliers,
                                       clip_scalar=clip_scalar,
                                       clip_method=remove_method,
                                       eeg_channels = eeg_channels,
                                       num_features=num_features,
                                       max_num_clips_for_pca=max_num_clips_for_pca,
                                       self=self)
            print('------------------')
        analyzed_directories.append(directory)
        print('------finished directory------')
print('------finished------')

Skipping the following directory: Other
--------------
There are 27 .rhd sessions in this directory!
mouse: b6_august_18_1
Analyzing file (1/27): b6_august_18_1075_plgt_190410_151142
The following session has already been analyzed, or is missing required files: b6_august_18_1075_plgt_190410_151142!
Analyzing file (2/27): B6_august_18_1100_plgt_190412_122841
The following session has already been analyzed, or is missing required files: B6_august_18_1100_plgt_190412_122841!
Analyzing file (3/27): B6_august_18_1100_plgt_190412_153929
The following session has already been analyzed, or is missing required files: B6_august_18_1100_plgt_190412_153929!
Analyzing file (4/27): b6_august_18_1100_plgt_190415_125812
The following session has already been analyzed, or is missing required files: b6_august_18_1100_plgt_190415_125812!
Analyzing file (5/27): b6_august_18_1100_plgt_190415_160655
The following session has already been analyzed, or is missing required files: b6_august_18_1100_plgt_190415_

[2019-05-02 16:34:22]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_105414.rhd
[2019-05-02 16:34:26]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_105914.rhd
[2019-05-02 16:34:29]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_110414.rhd
[2019-05-02 16:34:32]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_110915.rhd
[2019-05-02 16:34:36]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_111415.rhd
[2019-05-02 16:34:49]: Converting the fo

[2019-05-02 16:37:46]: Converting the following tetrode: 8!
[2019-05-02 16:37:46]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_104914.rhd
[2019-05-02 16:37:48]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_105414.rhd
[2019-05-02 16:37:52]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_105914.rhd
[2019-05-02 16:37:55]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_110414.rhd
[2019-05-02 16:37:59]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plg

ml-run-process ms4_geoff.sort --inputs raw_fname:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_j20n_76a_2/ParallelLinearGlobalTrack/b6_j20n_76a_2_1050_plgt_190501_104914_T6_raw.mda --outputs filt_out_fname:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_j20n_76a_2/ParallelLinearGlobalTrack/b6_j20n_76a_2_1050_plgt_190501_104914_T6_filt.mda firings_out:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_j20n_76a_2/ParallelLinearGlobalTrack/b6_j20n_76a_2_1050_plgt_190501_104914_T6_firings.mda pre_out_fname:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_j20n_76a_2/ParallelLinearGlobalTrack/b6_j20n_76a_2_1050_plgt_190501_104914_T6_pre.mda metrics_out_fname:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_j20n_76a_2/ParallelLinearGlobalTrack/b6_j20n_76a_2_1050_plgt_190501_104914_T6_metrics.json masked_out_fname:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_j20n_76a_2/ParallelLinearGlobalTrack/b6_j20n_76a_2_1050_plgt_190501_104914_T6_masked.mda --parameters freq_min:300 freq_max:7000 samplerate:24000 det

[2019-05-02 17:15:23]: Creating the following cut file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_104914_ms_4.cut!
[2019-05-02 17:15:24]: Converting the MountainSort output following filename to Tint: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_104914_T5_filt.mda
[2019-05-02 17:15:24]: Reading the spike data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_104914_T5_firings.mda
[2019-05-02 17:15:24]: Reading the spike data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_104914_T5_masked.mda
[2019-05-02 17:15:36]: Creating the following tetrode file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_76a_2\ParallelLinearGlobalTrack\b6_j20n_76a_2_1050_plgt_190501_104914_ms.5!
[2019-05-02 17

...

[2019-05-02 20:50:39]: Notch Filtering the EGF data!
[2019-05-02 20:51:02]: Creating the following EEG file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plgt_190501_101145_ms.eeg3!
[2019-05-02 20:51:02]: Filtering to create the EEG data!
[2019-05-02 20:51:03]: Notch Filtering the EEG data!
[2019-05-02 20:51:05]: Downsampling the EEG data to 250 Hz!
[2019-05-02 20:51:06]: Creating the following EGF file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plgt_190501_101145_ms.egf3!
[2019-05-02 20:51:06]: Filtering to create the EGF data!
[2019-05-02 20:51:07]: Notch Filtering the EGF data!
[2019-05-02 20:51:30]: Creating the following EEG file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plgt_190501_101145_ms.eeg4!
[2019-05-02 20:51:30]: Filtering to create the EEG data!
[2019-05-02 20:51:31]: Notch Filtering the EEG data!
[2019-05-02

[2019-05-02 21:00:30]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plgt_190501_133851.rhd
[2019-05-02 21:00:33]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plgt_190501_134351.rhd
[2019-05-02 21:00:36]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plgt_190501_134851.rhd
[2019-05-02 21:00:40]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plgt_190501_135352.rhd
[2019-05-02 21:00:53]: Converting the following tetrode: 5!
[2019-05-02 21:00:53]: Currently loading data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plg

[2019-05-02 21:09:36]: There was an error sorting the following file, consult terminal text file: /mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_j20n_82a_5/ParallelLinearGlobalTrack/b6_j20n_82a_5_1125_plgt_190501_132851_T2_raw.mda!#Red
ml-run-process ms4_geoff.sort --inputs raw_fname:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_j20n_82a_5/ParallelLinearGlobalTrack/b6_j20n_82a_5_1125_plgt_190501_132851_T3_raw.mda --outputs filt_out_fname:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_j20n_82a_5/ParallelLinearGlobalTrack/b6_j20n_82a_5_1125_plgt_190501_132851_T3_filt.mda firings_out:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_j20n_82a_5/ParallelLinearGlobalTrack/b6_j20n_82a_5_1125_plgt_190501_132851_T3_firings.mda pre_out_fname:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_j20n_82a_5/ParallelLinearGlobalTrack/b6_j20n_82a_5_1125_plgt_190501_132851_T3_pre.mda metrics_out_fname:/mnt/e/Apollo_D_Drive/data/VirtualMazeData/b6_j20n_82a_5/ParallelLinearGlobalTrack/b6_j20n_82a_5_1125_plgt_190501_

[2019-05-02 21:37:05]: The following position file already exists: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plgt_190501_132851.pos!
[2019-05-02 21:37:05]: Converting the MountainSort output following filename to Tint: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plgt_190501_132851_T1_filt.mda
[2019-05-02 21:37:05]: Reading the spike data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plgt_190501_132851_T1_firings.mda
[2019-05-02 21:37:05]: Reading the spike data from the following file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plgt_190501_132851_T1_masked.mda
[2019-05-02 21:37:17]: Creating the following tetrode file: E:\Apollo_D_Drive\data\VirtualMazeData\b6_j20n_82a_5\ParallelLinearGlobalTrack\b6_j20n_82a_5_1125_plgt_190501_132851_ms.1!
[2019-05

KeyError: '2'